In [1]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision import models
from PIL import Image
import pandas as pd

In [ ]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # 50% 확률로 이미지를 수평으로 뒤집음
    transforms.RandomVerticalFlip(),  # 50% 확률로 이미지를 수직으로 뒤집음
    transforms.RandomRotation(10),  # -10에서 10도 사이의 각도로 이미지를 회전
    transforms.ToTensor(),  # PIL 이미지나 numpy.ndarray를 pytorch의 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # RGB 채널별로 평균과 표준편차를 이용한 정규화
])

In [17]:
# 데이터 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),  # 50% 확률로 이미지를 수평으로 뒤집음
    transforms.RandomVerticalFlip(),  # 50% 확률로 이미지를 수직으로 뒤집음
    transforms.RandomRotation(10), 
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), 
])

# ResNet 모델 사용
model = models.resnet50(pretrained=True)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 3)

C:\Users\aims01\anaconda3\envs\metamatching\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\aims01\anaconda3\envs\metamatching\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [18]:

# 데이터 로딩
train_dataset = ImageFolder(root='train', transform=transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

# ResNet 모델 사용
# 클래스가 3개이므로 출력 뉴런의 수를 3개로 설정
best_loss = float('inf')  # 초기 최소 loss를 무한대로 설정

# GPU 사용 설정
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# 손실함수 및 옵티마이저 설정
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# 학습
num_epochs = 100
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    if loss.item() < best_loss:
        best_loss = loss.item()
        print(epoch)
        torch.save(model.state_dict(), 'best_model_resnet_retrained_norm.pth')
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}')

0
Epoch 1/100, Loss: 0.2728160321712494
Epoch 2/100, Loss: 0.5122842788696289
Epoch 3/100, Loss: 0.6162834167480469
Epoch 4/100, Loss: 1.2450895309448242
Epoch 5/100, Loss: 0.39911267161369324
Epoch 6/100, Loss: 1.0024333000183105
6
Epoch 7/100, Loss: 0.22754859924316406
Epoch 8/100, Loss: 0.560637891292572
8
Epoch 9/100, Loss: 0.09131034463644028
Epoch 10/100, Loss: 0.4976396858692169
Epoch 11/100, Loss: 0.22995133697986603
Epoch 12/100, Loss: 0.7342294454574585
Epoch 13/100, Loss: 0.4395310878753662
Epoch 14/100, Loss: 0.28216955065727234
Epoch 15/100, Loss: 0.30795592069625854
Epoch 16/100, Loss: 0.5097429752349854
Epoch 17/100, Loss: 0.17793108522891998
Epoch 18/100, Loss: 0.17655906081199646
Epoch 19/100, Loss: 0.1465073525905609
Epoch 20/100, Loss: 1.7558902502059937
20
Epoch 21/100, Loss: 0.042315009981393814
Epoch 22/100, Loss: 0.5798705816268921
Epoch 23/100, Loss: 0.9073683023452759
Epoch 24/100, Loss: 0.4546985328197479
Epoch 25/100, Loss: 0.20523792505264282
Epoch 26/100, L

"\nwith torch.no_grad():\n    for i in range(len(submission)):\n        img_path = submission.loc[i, 'path']\n        img = Image.open(img_path).convert('RGB')\n        img = transform(img).unsqueeze(0).to(device)\n\n        outputs = model(img)\n        _, predicted = torch.max(outputs.data, 1)\n        \n        # 모델이 각각의 동물로 예측하면 해당 동물 이름으로 분류\n        if predicted.item() == 0:  # 고양이로 예측한 경우\n            submission.loc[i, 'label'] = 'cat'\n        elif predicted.item() == 1:  # 개로 예측한 경우\n            submission.loc[i, 'label'] = 'dog'\n        else:  # 호랑이로 예측한 경우\n            submission.loc[i, 'label'] = 'tiger'\n\n# 예측 결과를 submission.csv에 저장\nsubmission.to_csv('submission.csv', index=False)\n"

In [22]:
dummy_input = torch.randn(1, 3, 640, 640).to(device)  # 모델의 입력 형태에 맞는 더미 데이터 생성
torch.onnx.export(model, dummy_input, 'resnet_retrained_model_augment_norm.onnx', opset_version=13)